<center> <h1>Ground motion prediction equations for the Chilean subduction zone</h1>
    
<center> <strong><em> Idini et a. 2016 </em></strong>

## Functional Form

$log_{10} Y = F_F (M_w, H, F_{eve}) + F_D (R, M_w, F_{eve}) + F_S$

$F_F$: contributions of the seismic source

$F_D$: the path effects between the seismic source and the site

$F_S$: local site effects

### Variables

$M_w$: Event Magnitude

$H$: Hypocentral depth $[km]$

$F_{eve}$: Dummy variable $[0$ for Interface; $1$ for Intraslab$]$

$R$: the source distance defined as the hypocentral distance for intraslab
and $M_w < 7.7$ interface earthquakes, and as the closest distance to rupture area for $M_w \geq 7.7$
interface earthquakes.

$T$: Period (s) $['PGA',0.01,0.02,0.03,...,4,5,7.5,10]$

$Vs_{30}$: Shear wave velocity for the first 30 meters of soil $[^m/_s]$

### Source ($F_F$)

$F_F (M_w, H, F_{eve}) = c_1 + c_2M_w + c_8*(H - h_0)*F_{eve} + \Delta f_m$

|$\Delta f_m$|$F_{eve}$|
|:-:|:-:|
|$c_9M_w^2$|$0$|
|$\Delta c_1 + \Delta c_2M_w$|$1$|

$h_0 = 50 km$

|**Event**|$F_{eve}$|
|:-:|:-:|
|**Interface**|$0$|
|**Intraslab**|$1$|

### Path ($F_D$)

$F_D ( R, M_w, F_{eve}) = g*log_{10}(R + R_0) + c_5R$

$R_o=(1-F_{eve})c_610^{c7*[Mw-Mr]}$

$g = c_3+c_4[M_w-M_r]+\Delta c_3F_{eve}$

$M_r = 5$

### Site ($F_S$)

#### Types of soils

|$S_C$|$T^*(s)$|
|:-:|:-:|
|$S_I$|**Not identifiable:** $HVSR \leq 2$|
|$S_{II}$|$T^* \leq 0.2$|
|$S_{III}$|$0.2\le T^* \leq0.4$|
|$S_{IV}$|$0.4\le T^* \leq0.8$|
|$S_{V}$|$T^* \ge 0.8$|
|$S_{VI}$|**Not identifiable: BB amplification or 2+peaks**|

$F_S = F_S(VS_{30}, T^*) = log_{10}(f_s) = S_{T^*} (T) log_{10}  (\frac {Vs_{30}}{Vref})$

$V_{ref} = 1530$  $[^m/_s]$

## Limits

|$M_w$ **range**|**Intraslab R limit (km)**|**Interface R limit (km)**|
|:-:|:-:|:-:|
|$5.5\leq M_w \leq 6.0$|$R \leq 150$|$R \leq 100$|
|$6.0\leq M_w \leq 6.5$|$R \leq 300$|$R \leq 150$|
|$6.5\leq M_w \leq 7.5$|$R \leq 300$|$R \leq 300$|
|     $M_w \ge 7.5$    |$R \leq 400$|$R \leq 400$|
|.........Limits.................|

In [2]:
import pandas as pd
import numpy as np

In [3]:
ci = pd.read_csv('C.csv',index_col=0)
s_t = pd.read_csv('s_t.csv',index_col=0)

In [4]:
ci.loc['PGA']

c3    -0.97558
c5    -0.00174
Dc3   -0.52745
rr     0.23200
c1    -2.85480
c2     0.77410
c9    -0.03958
c8     0.00586
Dc1    2.56990
Dc2   -0.47610
re     0.17200
rt     0.28900
Name: PGA, dtype: float64

In [5]:
def FF(T,Mw,H,Feve):
    import pandas as pd
    ho=50
    ci = pd.read_csv('C.csv',index_col=0)
    c_i = ci.loc[T]
    if Feve==0:
        dfm=c_i['c9']*(Mw**2)
    elif Feve==1:
        dfm=c_i['Dc1']+c_i['Dc2']*Mw
    FF = c_i['c1']+c_i['c2']*Mw+c_i['c8']*(H-ho)*Feve+dfm
    return FF

In [15]:
FF('PGA',7,30,1)

1.6839000000000004

In [7]:
def FD(T,R,Mw,Feve):
    import pandas as pd
    import numpy as np
    Mr=5.0;c4=0.1;c6=5;c7=0.35
    ci = pd.read_csv('C.csv',index_col=0)
    c_i = ci.loc[T]
    Ro = (1-Feve)*c6*10**(c7*(Mw-Mr))
    g = c_i['c3']+c4*(Mw-Mr)+c_i['Dc3']*Feve
    FD = g*np.log10(R+Ro)+R*c_i['c5']
    return FD

In [8]:
FD('PGA',50,7.8,1)

-2.1648912844030814

In [9]:
def FS(Vs30,Ts,T):
    import pandas as pd
    import numpy as np
    Vref=1530
    st=pd.read_csv('s_t.csv',index_col=0)
    s_i = st.loc[T]
    if Ts == 0:
        si = s_i['sI']
    elif 0 < Ts and Ts <= 0.2:
        si = s_i['sII']
    elif 0.2 < Ts and Ts <= 0.4:
        si = s_i['sIII']
    elif 0.4 < Ts and Ts <= 0.8:
        si = s_i['sIV']
    elif Ts < 0.8:
        si = s_i['sV']
    else:
        si = s_i['sVI']
    FS = si*np.log10(Vs30/Vref)
    return FS

In [22]:
def Idini_2016(T,Mw,H,R,Feve,Vs30,Ts):
    ff = FF(T,Mw,H,Feve)
    fd = FD(T,R,Mw,Feve)
    fs = FS(Vs30,Ts,T)
    return 10**(ff+fd+fs)

In [26]:
Idini_2016('PGA',7.8,30,50,1,400,0.6)

0.6620877345971488